# 关联规则

机器学习领域有一个经典的啤酒和尿布的故事：

有一家超市，在分析用户的购买数据时，发现一个有趣的例子：有相当数量买尿布的顾客，同时还买了啤酒，于是经过进一步观察和分析，他们发现不少男性顾客为家庭承担了购买尿布的任务，买了尿布后觉得来都来了，也就买些啤酒犒劳下自己。于是这家超市尝试着在尿布货架旁边放啤酒，果然卖出了更多的啤酒。

这样的例子在电商平台更明显，我们在京东把一件商品加入购物车，紧接着就会看到“购买了该商品的用户还购买了”，以及“您可能还需要”这些推荐。有部分推荐靠人工定下的规则就可以判断，譬如你买了牙刷就很可能需要牙膏，买了洗发水就很可能需要护发素。

但是像天猫京东这样的大电商平台上通常有几千万个货物，这些货物之间存在的潜在联系，不可能单靠人脑推理得出，特别是很多联系会由季节乃至突发事件决定。因此我们在电商平台购物时，被推荐的商品都是靠推荐算法和系统找到的。

### 名词解释

将每件商品作为列，对购物清单重新编码的方式，称为**独热编码 (One Hot Encoding)**，这个编码方式在数据分析、机器学习和自然语言处理的很多领域都有用到。此外，还有一些相关的重要概念：

**商品组合：**一个商品组合包含一个以上的产品。

**关联规则：**如果一个商品组合 A 和另一个商品组合 B 同时出现在一个或者多个顾客的购物清单上，那么我们就说这两个商品组合中存在关联规则。

如果我们要研究买了 A 的顾客再买 B 的情况，这样的规则就表示为 A→B，→ 之前的称为 前导，→ 之后的称为 后继。

注意：A→B 和 B→A 是两条不一样的规则。

**支持度(Support)：**某个商品组合出现的次数与总次数之间的比例。

譬如前面我们已经知道了，10 个顾客的购物清单中出现了 4 次可乐，所以“可乐”的支持度是 4/10 = 0.4。出现了 3 次“可乐 + 啤酒”，所以“可乐 + 啤酒”的支持度是 3/10 = 0.3。

**置信度(Confidence)：**购买了商品 A 的顾客中，有多大比例购买商品 B。

4 位顾客购买了可乐，其中 3 位买了啤酒，因此我们可以说存在一条从可乐到啤酒的关联规则，这条规则的置信度是 3/4 = 0.75。写作：置信度(可乐 → 啤酒) = 0.75。

**提升度(Lift)：**购买了商品 A 的顾客，相比于一般顾客，多大程度更倾向于购买商品 B。

提升度是区别于置信度的，我们已经知道置信度(可乐 → 啤酒)是 0.75，虽然数值挺高的，但却不能完全反映购买可乐的顾客更倾向于购买啤酒。我们从独热编码后的购物清单中可以看到总共 10 名顾客中有 7 名购买了啤酒（也就说“啤酒”自身的支持度就是 0.7），这个比例的确比 0.75 小一点，但是算不上很明显。

同时我们还可以看到置信度(可乐 → 面包)也是 0.75，但是在所有顾客中购买面包的比例是 90%（支持度为 0.9）。那么显然，购买可乐的顾客相比于购买面包，更倾向于购买啤酒。

所以 提升度 这个指标，就是要将 (A→B) 的置信度和 B 的支持度进行对比，所以对应的计算公式是：提升度(A→B) = 置信度(A→B) / 支持度(B)。

**杠杆率(Leverage)：**杠杆率(A→B) = 置信度(A→B) − 支持度(A) × 支持度(B)。

杠杆率越大，A 和 B 的关系越密切，当杠杆率为 0 时说明 A 和 B 互相独立。

**确信度(Conviction)：**确信度(A→B) = (1 − 支持度(B)) / (1 − 置信度(A→B))。

确信度也用来衡量 A 和 B 的独立性，值越大，A 和 B 的关联性也越强。

### 实例测试

In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

# 在这个数据结构里，每一列代表一件商品是否被顾客购买，每一行是一名顾客在每件商品上是否有购买。
dataset = [
  ["面包", "牛奶"],
  ["面包", "尿布", "啤酒", "鸡蛋"],
  ["牛奶", "尿布", "啤酒", "可乐"],
  ["面包", "牛奶", "尿布", "啤酒"],
  ["面包", "牛奶", "尿布", "可乐"],
  ["面包", "牛奶"],
  ["面包", "可乐", "啤酒", "牛奶"],
  ["牛奶", "面包", "啤酒", "可乐"],
  ["面包", "牛奶", "尿布", "啤酒"],
  ["面包", "啤酒", "尿布", "尿布"]
]

te = TransactionEncoder()
encorder = te.fit(dataset)
one_hot_encoded_array = encorder.transform(dataset)

df = pd.DataFrame(one_hot_encoded_array, columns=encorder.columns_)
print(df)

      可乐     啤酒     尿布     牛奶     面包     鸡蛋
0  False  False  False   True   True  False
1  False   True   True  False   True   True
2   True   True   True   True  False  False
3  False   True   True   True   True  False
4   True  False   True   True   True  False
5  False  False  False   True   True  False
6   True   True  False   True   True  False
7   True   True  False   True   True  False
8  False   True   True   True   True  False
9  False   True   True  False   True  False


In [2]:
from mlxtend.frequent_patterns import apriori, association_rules

# 调用 apriori() 函数来寻找满足最小支持度的商品组合。
# 1.找出支持度大于 0.4 的商品，仅对它们进行数据分析
frequent_itemsets = apriori(df, min_support=0.4, use_colnames=True)

# 2.找到置信度大于 0.8 的关联规则
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.8)

# 3.显示满足条件的规则，按照提升度从高向低排序
print(rules.sort_values("lift", ascending=False))

  antecedents consequents  antecedent support  consequent support  support  \
0        (可乐)        (牛奶)                 0.4                 0.8      0.4   
1        (尿布)        (啤酒)                 0.6                 0.7      0.5   
6    (面包, 尿布)        (啤酒)                 0.5                 0.7      0.4   
4        (牛奶)        (面包)                 0.8                 0.9      0.7   
2        (啤酒)        (面包)                 0.7                 0.9      0.6   
3        (尿布)        (面包)                 0.6                 0.9      0.5   
5    (啤酒, 尿布)        (面包)                 0.5                 0.9      0.4   
7    (啤酒, 牛奶)        (面包)                 0.5                 0.9      0.4   

   confidence      lift  leverage  conviction  
0    1.000000  1.250000      0.08         inf  
1    0.833333  1.190476      0.08         1.8  
6    0.800000  1.142857      0.05         1.5  
4    0.875000  0.972222     -0.02         0.8  
2    0.857143  0.952381     -0.03         0.7  
3    0.83

只有 3 条规则的提升度超过 1，能够用于推荐：

1、可乐 → 牛奶 这一条关联规则的支持度是 0.4，置信度是 1，提升度是 1.25，所以向购买可乐的顾客再推荐牛奶，是一个有意义的举措。

2、尿布 → 啤酒 的支持度是0.5，置信度 0.83，提升度是 1.19，也说明可以向购买尿布的顾客推荐啤酒。

3、(尿布,面包)→ 啤酒 这条关联规则，支持度 0.4，置信度 0.8，提升度和 尿布 → 啤酒 对比一下，我们可以看出两点：

两个规则的后继都是啤酒，而先导条件(尿布,面包)比尿布更加具体，也就是要求更高；
同时，我们会发现，更高的要求却没有带来更高的置信度。也就是说发现一个既买了尿布，又买了面包的顾客再向他推荐啤酒，还不如一看到有人买尿布不用管他有没有买面包就直接推荐啤酒，所以这条规则没有带来额外的价值，是冗余的。
所以有这样一个判读规则是否冗余的标准：如果一个更具体的关联规则的置信度比其一般规则的更低，那具体的关联规则为冗余规则，没有必要保留。